# Voedselprijzen in de derde wereld

Robert-Jan Korteschiel (10399143)  
Robert Houten  
Sander Kohnstamm (10715363)  
Joost de Wildt (12173002)  


## Vooronderzoek

Dit is een dataset die wekelijks wordt geupdated over voedselprijzen. De auteur is het World Food Programme, een humanitaire organisatie die steeft naar het ideaal van "zero hunger". Daar baseren ze al hun werk al op data en hebben ook al goede dataviz draaien. Interessant is dat ze hier combineren met veel GIS informatie zoals administratieve grenzen, wegennetwerken en klimaatdata. 

<div style="display: flex; width: 100%; justify-content: space-around;">
    <div style="width: 45%">
        <img src="food_viz.png" style="display: block; width: 100%; height: auto;" alt="GIS visualisation">
    </div>
    <div style="width: 45%">
        <img src="patterns.jpg" style="display: block; width: 100%; height: auto;" alt="Line visualisation">
    </div>
</div>


Er is dus al veel werk gedaan op de data, maar er mist ook veel informatie. Opvallend is dat economische en politieke gegevens missen in de analyse. Daarnaast bekijken ze de derde wereld echt per land en zien kijken ze bijna niet naar hoe deze problematiek grensoverstijgend is (of is dat wel zo?). Bovendien is het alles behalve een verhaal, dit is echt een visualisatie. Het is interessant of we delen kunnen namaken en tot een verhaal kunnen omvormen. 

## Invalshoeken

1. Welke van de volgende gebeurtenissen heeft de grootste invloed op globale voedselprijzen?
    - Temperatuur
    - Brandstofprijs
    - Dagloon  


2. Vallen gebeurtenissen te herleiden uit voedselprijzen?



## Vergelijkbaarheid per commodity per land

De vergelijkbaarheid per commodity per land echt heel laag, maar hele korte stukken data overlappen echt. Het WFP meet blijkbaar alleen als ze geinteresseerd zijn in iets, om de een of andere reden. Vervolgens redeneren we dat we al het voedsel samentrekken tot een enkele trend, dat moet de vergelijkbaarheid al aardig hoger maken. Voor onze onderzoeksvraag is het immers alleen interessant dat we veranderingen in voedselprijzen afzetten tegen brandstof, lonen en klimaat. Misschien dat we daarna gewoon een land kiezen met het meeste data, los van de vergelijkbaarheid, voor de andere onderzoeksvraag.


# Dataverwerking

## Bootstrap

In [5]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sklearn import preprocessing
from operator import itemgetter, attrgetter
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

init_notebook_mode(connected=True)

pd.options.display.max_rows = 100
pd.options.display.max_seq_items = 100

## Inladen

en laat de head() zien

In [2]:
food_df = pd.read_csv("./food_data/food.csv", low_memory=False)
display(food_df.head())

climate_df = pd.read_csv("./climate_data/GlobalLandTemperaturesByCountry.csv", low_memory=False)
display(climate_df.head())


,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


# Processing

## Normalisatie

Normalisatie functie voor verschillende groepen in de data. Ik vermoed dat je het beste binnen een land op commodity kan normaliseren. Doe je het op een andere plek dan worden minder waardevolle commodities binnen groepen minder belangrijk. Het is aannemelijk dat graan wholesale flink goedkoper is dan graan retail. Als je die in absolute nummers optelt dan domineert het verschil in de retailprijs.

Je zou overigens nog een argument kunnen maken dat je eigenlijk op provincie of zelfs stadsniveau moet gaan normaliseren.


In [6]:
def normalize(group_df):
    # bootstrap a new normalizer   
    min_max_scaler = preprocessing.MinMaxScaler()
    
    # reshape the series to array so that the normalizer accepts it
    to_normalize = group_df["mp_price"].values.reshape(-1, 1)
    
    # do the actual normalisation     
    x_scaled = min_max_scaler.fit_transform(to_normalize)
    
    # undo some weird numpy nesting of arrays    
    x_scaled = np.concatenate(x_scaled).ravel()
    
    # concatenate it to the group_df    
    group_df["mp_price_norm"] = x_scaled
    
    # trow the group out     
    return group_df
    
food_norm_df = food_df.groupby(by=["adm0_name", "cm_name"]).apply(normalize)
# display(food_norm_df[food_norm_df["adm0_name"] == "Somalia"])

## Groeperen en verwerken

Groepeert alle commodities op basis het het eerste woord in hun definitie. Daarna berekent hij het gemiddelde en het verschil per jaar. 

**LET OP! Hier zit een grote assumptie. Een jaar kan een waarde krijgen op een enkele meting**

In [9]:
# groepeer de commodities met hun eerst woord
pat = '^([\w\-]+)'
selection = food_norm_df["cm_name"].str.extract(pat, expand=False)
grouped_commodity_df = food_norm_df.groupby(by=["adm0_name", selection, "mp_year"])

# bereken het gemiddelde per jaar en vergelijk dat met het vorige jaar
grouped_commodity_mean_df = grouped_commodity_df["mp_price_norm"].mean()

# unstack zodat het verschil per jaar berekend kan worden
grouped_commodity_diff_df = grouped_commodity_mean_df.unstack(level=2).diff(axis=1)
grouped_commodity_diff_df

mp_year                                 1992  1993  1994  1995  1996     1997  \
adm0_name                   cm_name                                             
Afghanistan                 Bread        NaN   NaN   NaN   NaN   NaN      NaN   
                            Exchange     NaN   NaN   NaN   NaN   NaN      NaN   
                            Fuel         NaN   NaN   NaN   NaN   NaN      NaN   
                            Livestock    NaN   NaN   NaN   NaN   NaN      NaN   
                            Rice         NaN   NaN   NaN   NaN   NaN      NaN   
                            Wage         NaN   NaN   NaN   NaN   NaN      NaN   
                            Wheat        NaN   NaN   NaN   NaN   NaN      NaN   
Algeria                     Apples       NaN   NaN   NaN   NaN   NaN      NaN   
                            Bananas      NaN   NaN   NaN   NaN   NaN      NaN   
                            Beans        NaN   NaN   NaN   NaN   NaN      NaN   
                            Bread        NaN   NaN   NaN   NaN   NaN      NaN   
                            Carrots      NaN   NaN   NaN   NaN   NaN      NaN   
                            Cheese       NaN   NaN   NaN   NaN   NaN      NaN   
                            Eggs         NaN   NaN   NaN   NaN   NaN      NaN   
                            Fish         NaN   NaN   NaN   NaN   NaN      NaN   
                            Fuel         NaN   NaN   NaN   NaN   NaN      NaN   
                            Lentils      NaN   NaN   NaN   NaN   NaN      NaN   
                            Meat         NaN   NaN   NaN   NaN   NaN      NaN   
                            Milk         NaN   NaN   NaN   NaN   NaN      NaN   
                            Oil          NaN   NaN   NaN   NaN   NaN      NaN   
                            Onions       NaN   NaN   NaN   NaN   NaN      NaN   
                            Oranges      NaN   NaN   NaN   NaN   NaN      NaN   
                            Pasta        NaN   NaN   NaN   NaN   NaN      NaN   
                            Potatoes     NaN   NaN   NaN   NaN   NaN      NaN   
                            Rice         NaN   NaN   NaN   NaN   NaN      NaN   
                            Sugar        NaN   NaN   NaN   NaN   NaN      NaN   
                            Tea          NaN   NaN   NaN   NaN   NaN      NaN   
                            Tomatoes     NaN   NaN   NaN   NaN   NaN      NaN   
Angola                      Beans        NaN   NaN   NaN   NaN   NaN      NaN   
                            Cassava      NaN   NaN   NaN   NaN   NaN      NaN   
                            Fish         NaN   NaN   NaN   NaN   NaN      NaN   
                            Maize        NaN   NaN   NaN   NaN   NaN      NaN   
                            Oil          NaN   NaN   NaN   NaN   NaN      NaN   
                            Rice         NaN   NaN   NaN   NaN   NaN      NaN   
                            Salt         NaN   NaN   NaN   NaN   NaN      NaN   
                            Sugar        NaN   NaN   NaN   NaN   NaN      NaN   
Armenia                     Apples       NaN   NaN   NaN   NaN   NaN      NaN   
                            Bread        NaN   NaN   NaN   NaN   NaN      NaN   
                            Cabbage      NaN   NaN   NaN   NaN   NaN      NaN   
                            Carrots      NaN   NaN   NaN   NaN   NaN      NaN   
                            Cheese       NaN   NaN   NaN   NaN   NaN      NaN   
                            Cucumbers    NaN   NaN   NaN   NaN   NaN      NaN   
                            Eggs         NaN   NaN   NaN   NaN   NaN      NaN   
                            Exchange     NaN   NaN   NaN   NaN   NaN      NaN   
                            Fish         NaN   NaN   NaN   NaN   NaN      NaN   
                            Fuel         NaN   NaN   NaN   NaN   NaN  0.06372   
                            Lentils      NaN   NaN   NaN   NaN   NaN      NaN   
                            Mea

## Data selecteren 

Verkenning en functies om te selecteren.

Spoiler: De groep "Rice" heeft de meeste datapunten en 2016 is het jaar met de minste NaN's.

In [10]:
# een telling van hoeveelheid gevulde vakjes per commodity
years_with_data_per_country = [(commodity_type, grouped_commodity_diff_df.xs(commodity_type, level=1, drop_level=False).count().sum()) for commodity_type in selection.unique()]

# welke is het beste
uselection_sorted = sorted(years_with_data_per_country, key=itemgetter(1), reverse=True)

# maak het even makkelijk voor de visualisatie
uselection_sorted_comm = [comm[0] for comm in uselection_sorted]
uselection_sorted_comm_ten = uselection_sorted_comm[:10]
uselection_sorted_comm_ten


['Rice',
 'Maize',
 'Oil',
 'Wheat',
 'Beans',
 'Sugar',
 'Sorghum',
 'Meat',
 'Millet',
 'Potatoes']

Een utility functie die de kaart interactief gaat maken! Selectie wordt heel makkelijk.

In [11]:
def select_commodity(commodity, year):
    commodity_series = grouped_commodity_diff_df.xs(commodity, level=1, drop_level=False)[year]
    commodity_index = commodity_series.index.get_level_values(0)
    commodity_values = commodity_series.values
    return [commodity_index, commodity_values]


# Visualisatie: Maps

## Food change maps


In [12]:

@interact
def interact(food=uselection_sorted_comm_ten, year=(2010, 2019, 1)):
    # Plot the mean price, of all commodities on map, of all years.
    com_index_value_pair = select_commodity(food, year)

    data = [go.Choropleth(
        locations = com_index_value_pair[0],
        locationmode = "country names",
        z = com_index_value_pair[1],
    )]

    fig = go.Figure(data = data)
    iplot(fig)
    


interactive(children=(Dropdown(description='food', options=('Rice', 'Maize', 'Oil', 'Wheat', 'Beans', 'Sugar',…

# Bronnen
[datasource] https://data.humdata.org/dataset/wfp-food-prices  
[organisation goals] https://www1.wfp.org/zero-hunger  
[data description] https://docs.wfp.org/api/documents/WFP-0000040024/download/  
[dataviz platform] https://dataviz.vam.wfp.org  
[dataviz hunger] https://dataviz.vam.wfp.org/Hunger-Analytics-Hub#  
[WFP price] https://dataviz.vam.wfp.org/economic_explorer/price-forecasts-alerts   
[VN prijzen] https://treasury.un.org/operationalrates/OperationalRates.php  
[dataset omschrijving] http://mvam.org/2018/11/20/getting-up-to-speed-wfp-food-data-on-hdx/  
[interactieve plots] https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6  
[GDELT database] https://www.gdeltproject.org/  

